# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
import glob
import os

def df_names_us():
    path = os.path.abspath(os.getcwd()) + r'\names'
    all_files = glob.glob(path + "/*.txt")
    df = pd.DataFrame()
    for filename in all_files:
        sub_df = pd.read_csv(filename, names=['name', 'gender', 'births'])
        sub_df.insert(0, 'year', filename[-8:-4])
        df = df.append(sub_df, ignore_index=True)
    df['year'] = df['year'].astype(int)
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    df.reset_index(inplace=True, drop=True)
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [3]:
# names fr

def df_names_fr():
        sub_df_fr = pd.read_csv('names/nat2020.csv', sep=";", header=0, keep_default_na=False)
        df_extracted = sub_df_fr
        df_extracted['sexe'].replace([1, 2], ['M', 'F'], inplace=True)
        df_extracted.columns = ['gender', 'name', 'year', 'births']
        df_extracted = df_extracted.drop(df_extracted[df_extracted.year == 'XXXX'].index)
        df_extracted['year'].astype(int)
        df_extracted = df_extracted.drop(df_extracted[df_extracted.name == '_PRENOMS_RARES'].index)
        df_extracted['name'] = df_extracted['name'].str.capitalize()
        df_extracted = df_extracted.drop(df_extracted[df_extracted.name.str.len() < 2].index)
        df_extracted = df_extracted[['year', 'name', 'gender', 'births']]
        df_extracted = df_extracted.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
        df_extracted.reset_index(inplace=True, drop=True)
        return df_extracted

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [4]:
# taux de change
def df_taux_change(devises):
    df = pd.read_csv('Webstat_Export_20210929.csv', sep=";")
    df['Titre :'] = pd.to_datetime(df['Titre :'], format='%d/%m/%Y', errors='coerce')
    df= df.rename(columns={'Titre :':'Date'})
    df.dropna(inplace=True)
    df = df.set_index('Date')
    df = df.replace(',','.', regex=True)
    df.rename(columns=lambda x: x[x.find("(")+1:x.find(")")], inplace=True)  
    df = (df.drop(devises, axis=1)
         .join(df[devises].apply(pd.to_numeric, errors='coerce')))
    df = df[df[devises].notnull().all(axis=1)]
    df[devises].astype(float)
    return df[devises]

### Tests

In [5]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [6]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [7]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.137s

OK
